### Task 1 - Create data set 

In [114]:
import numpy as np
from scipy.integrate import solve_ivp

# Define the Lorenz system
def lorenz(t, xyz, sigma, r, b):
    x, y, z = xyz
    dxdt = -sigma*x + sigma*y
    dydt = -x*z + r*x - y
    dzdt = x*y - b*z
    return [dxdt, dydt, dzdt]

# Set the parameters
sigma = 10
r = 28
b = 8/3

# Set the initial conditions and time span
t0, tf = 0, 100
x0 = [1, 1, 1]
t_span = [t0, tf]

# Define the time step and the number of steps
dt = 0.01
t_steps = int((tf - t0) / dt) + 1

# Solve the system using the fourth-order Runge-Kutta method
sol = solve_ivp(lorenz, t_span, x0, t_eval=np.linspace(t0, tf, t_steps), args=(sigma, r, b))

Y = sol.y
nComp = np.shape(Y[1])
print(np.shape(Y)[1])
print(nComp)


10001
(10001,)


### Task 2

In [115]:
class Reservoir:
    def __init__(self, input_size, nReservoirNeurons, sigma, Y):
        self.n = input_size
        self.N = nReservoirNeurons
        self.sigma = sigma
        self.Y = Y
        self.T = Y.shape[1]
        self.w_in = np.random.normal(loc=0, scale=sigma, size=(self.N, self.n))
        self.w_reserv = self.get_reservoir_weights()
        self.w_out = np.random.normal(loc=0, scale=sigma, size=(self.n, self.N))
        
    def get_reservoir_weights(self):
        sparsity = 0.1
        dense_w = np.random.normal(loc=0, scale=sigma, size=(self.N, self.N))
        frac = int(self.N*self.N*sparsity)
        indices = np.random.choice(self.N*self.N, frac, replace=False)
        
        row_indices = indices // self.N  # convert 1D indices to 2D row indices
        col_indices = indices % self.N   # convert 1D indices to 2D column indices
        sparse_w = np.zeros((self.N, self.N))
        
        sparse_w[row_indices, col_indices] = dense_w[row_indices, col_indices]
        return sparse_w
        
    def g(self, local_field):
        return np.tanh(local_field)
    
    
    def compute_reservoir_states(self):
        r = np.zeros(self.N)
        R = np.zeros((self.N, self.T))
        
        for t in range(self.T-1):
            reservoir_field = np.dot(self.w_reserv, r)
            input_field = np.dot(self.w_in, self.Y[:,t])
            r = self.g(reservoir_field + input_field)
            R[:, t] = r
            
        alpha = 1e-6
        I = np.identity(self.N)
        self.w_out = np.dot(np.dot(self.Y, R.T), np.linalg.inv(np.dot(R, R.T) + alpha * I))
        
    def predict(self, x0):
        x = x0
        r = np.zeros(self.N)
        y = np.zeros((self.n, self.T))
        y[:,0] = x0
        for t in range(self.T-1):
            reservoir_field = np.dot(self.w_reserv, r)
            input_field = np.dot(self.w_in, x)
            r = self.g(reservoir_field + input_field.reshape(self.N))
            y[:, t+1] = np.dot(self.w_out, r)
            x = y[:, t+1]
        return y

In [123]:
#Parameters
reserv_size = 1000
sigma = 0.2
temp = Y[1,:]
x2 = temp.reshape((1, Y.shape[1]))

reservoir_3D = Reservoir(input_size=3, nReservoirNeurons=reserv_size, sigma=sigma, Y=Y)
reservoir_3D.compute_reservoir_states()

reservoir_1D = Reservoir(input_size=1, nReservoirNeurons=reserv_size, sigma=sigma, Y=x2)
reservoir_1D.compute_reservoir_states()

### Task 3